In [ ]:
! which python
import oracledb
import random

In [ ]:
# Connection details
db_host = ''
db_port = 1526
db_sid = ''
db_usr = ''
db_pwd = ''
table = ''

In [ ]:
# Database connection setup
dsn = oracledb.makedsn(db_host, db_port, db_sid)
connection = oracledb.connect(user=db_usr, password=db_pwd, dsn=dsn)
connection.autocommit = False # Manage commits manually for better performance
cursor = connection.cursor()
print('Client configured')

In [ ]:
# Specify batch size
cursor.arraysize = 10000

# Set the number of updates to do
number_of_updates = 15000

In [ ]:
# Fetch ID of a random subset of records
sql_fetch_keys = f"""
      select PK from {table} FETCH FIRST :num ROWS ONLY
"""
cursor.execute(sql_fetch_keys, num=number_of_updates)
keys_to_update = [(row[0],) for row in cursor.fetchall()]

In [ ]:
# Print sample PKs
sample_pks = [val[0] for val in random.sample(keys_to_update, min(20, len(keys_to_update)))]
print(f'Sample PKs: {sample_pks}')

In [ ]:
# Update records in batches for better performance
sql_update_query = f"update {table} set MODIFIEDTS = SYSTIMESTAMP where PK = :1"
cursor.prepare(sql_update_query)
batch_errors = cursor.executemany(None, keys_to_update, batcherrors=True)

In [ ]:
# Commit changes
connection.commit()
print('Changes applied')

In [ ]:
# Check errors
if batch_errors:
    for error in batch_errors:
        print("Error occurred at row offset", error.offset, ", error message is:", error.message)
else:
    print('No errors occured')

In [ ]:
# Print updated sample entities
in_part = f"({','.join(str(pk) for pk in sample_pks)})"
sql_fetch_updates = f"select PK, MODIFIEDTS from {table} where PK in {in_part}"

print(sql_fetch_updates)
cursor.execute(sql_fetch_updates)
result = cursor.fetchall()
print('\n'.join(str(tpl) for tpl in result))

In [ ]:
# Clean up
cursor.close()
connection.close()